In [1]:
import json
from lxml import etree
import requests
import re
from multiprocessing import Queue
import json

In [2]:
ini_url= "http://college.gaokao.com/spepoint/y2016/"
xpath1 = '//*[@id="wrapper"]/div[1]/p[3]/*[starts-with(@href, "http://college.gaokao.com/spepoint/y")]//@href'
xpath2= '//*[@id="wrapper"]/div[4]/ul[4]/a//@href'
forRe = '\[\'(http://college.gaokao.com/spepoint/y\d+/p)(\d+)'
posturl = 'http://211.87.230.125/10/GetResource '

data = { 'result': []}

ini_r = requests.get(ini_url)
ini_html = ini_r.text
ini_s = etree.HTML(ini_html)
ini_content = ini_s.xpath(xpath1)

queueYear = Queue()
queueLastpage = Queue()
queueAllpage = Queue()

for i in ini_content:
    queueYear.put(i)

while not queueYear.empty():
    r1 = requests.get(queueYear.get())
    html1 = r1.text
    s1=etree.HTML(html1)
    content1 = s1.xpath(xpath2)
    if content1:
        queueLastpage.put(content1)
        #print(content1)

while not queueLastpage.empty():
    thaturl = str(queueLastpage.get())
    matchO = re.match(r''+forRe,thaturl, re.M|re.I)
    num = int(matchO.group(2))
    thisurl = matchO.group(1)
    for i in range(1, num):
        queueAllpage.put((thisurl+str(i)))

while not queueAllpage.empty():
    data['result'].append({'url':queueAllpage.get()})
    

    
json_str = json.dumps(data,indent=4,ensure_ascii=False)   

headers = {'Content-Type': 'application/json'}
#request = requests.post(posturl, data=json_str.encode('utf-8'), headers=headers)

print(json_str)


{
    "result": []
}


In [5]:
json_str = json.dumps(data,indent=4,ensure_ascii=False)
headers = {'Content-Type':'application/json'}
request = requests.post("http://211.87.230.125/10/GetResource", data=json_str.encode('utf-8'), headers=headers)
    

In [ ]:
from lxml import etree
import requests
import re
from queue import Queue
import json
import urllib.request


ini_url= "http://college.gaokao.com/spepoint/y2016/"
xpath1 = '//*[@id="wrapper"]/div[1]/p[3]/*[starts-with(@href, "http://college.gaokao.com/spepoint/y")]//@href'
xpath2= '//*[@id="wrapper"]/div[4]/ul[4]/a//@href'
forRe = '\[\'(http://college.gaokao.com/spepoint/y\d+/p)(\d+)'
posturl = 'http://211.87.230.125/10/GetResource '

data = { 'result': []}

ini_r = requests.get(ini_url)
ini_html = ini_r.text
ini_s = etree.HTML(ini_html)
ini_content = ini_s.xpath(xpath1)

queueYear = Queue()
queueLastpage = Queue()
queueAllpage = Queue()

#print(ini_content)

for i in ini_content:
    queueYear.put(i)

while not queueYear.empty():
    r1 = requests.get(queueYear.get())
    html1 = r1.text
    s1=etree.HTML(html1)
    content1 = s1.xpath(xpath2)
    if content1:
        queueLastpage.put(content1)
        #print(content1)

while not queueLastpage.empty():
    thaturl = str(queueLastpage.get())
    matchO = re.match(r''+forRe,thaturl, re.M|re.I)
    num = int(matchO.group(2))
    thisurl = matchO.group(1)
    for i in range(1, num):
        queueAllpage.put((thisurl+str(i)))

while not queueAllpage.empty():
    data['result'].append({'url':queueAllpage.get()})

def send():
    values = urllib.parse.urlencode(data).encode(encoding='UTF8')
    headers = {'Content-Type': 'application/json'}
    request = urllib.request.Request(url=posturl, headers=headers, data=json.dumps(data).encode())
    response = urllib.request.urlopen(request)
    print(response)

send()